# 词的表示

此内嵌常用于 NLP，即把一组词映射为一组向量，前面提到的 one-hot 就是其中的一种，one-hot 的缺点是不能表示词之间的相似性。在任何给定的语料中，我们计算`(cat, dog)(knife, spoon)` 之间的相似度是通过点乘来计算，但 one-hot 计算出的结果总是 0.

常见的方法是基于 IR（information retrieval），如 TF-IDF、LSA（latent semantic Analysis）、topic modeling等。

**分布式表示**

如下示例：
```
Pairs is the capital of France.
Berlin is the capital of Germany.
```
不需要英语知识我们就可以知道，Pairs 与 Berlin、France 与 Germany 是类似的。而分布式表示即使寻找一个转换函数 $\phi$ ，来转换每个词与它相关的向量，如下：
$$
\large{
\begin{split}
\phi(Pairs) - \phi(France) \approx \phi(Berlin) - \phi(Germany)
\end{split}
}
$$

# word2vec

常见的架构有两种
- Continuous Bag of Words（CBOW）
- Skip-gram

经过实现 CBOW 更快。

## skip-gram

他是基于中心词的，如下：
```
I love green eggs and ham
```
假设 窗口的大小为 3 ，那么上面的句子可以切分成如下格式：
```
([I, green], love)
([love, eggs], green)
([green, and], eggs)
...
```
根据中心词，我们可以将处理的数据集翻译成 `(input, output)` (input 就是中心词，output 就是上面方括号里面的词)，那么上面的示例就可以写成如下形式：
```
(love, I), (love, green), (green, love), (green, eggs), (eggs, green), (eggs, and), ...
```
上面的数据可以当做正例，之后在随机的生成一些负例：
```
(love, Sam), (love, zebra), (green, thing),...
```

这样就可以组成我们的训练数据集：
```
((love, I), 1), ((love, green), 1), ... ((love, Sam), 0), ((love, zebra), 0),..
```

示意图如下：
![image](https://ws4.sinaimg.cn/large/69d4185bly1fyfdaqwks1j20c00bqmxp.jpg)

Keras 可以实现，假设词汇表大小为 5000，词内嵌大小为 300，窗口大小为 1.

In [1]:
from keras.layers import Merge
from keras.layers.core import Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Sequential
vocab_size = 5000
embed_size = 300

Using TensorFlow backend.


In [ ]:
# word
word_model = Sequential()
word_model.add(Embedding (vocab_size, embed_size,
                          embeddings_ initializer="glorot_uniform",
                          input_length=1) )
word_model.add(Reshape( (embed_size, ) ) )

# context words
context_model = Sequential()
context_model.add(Embedding (vocab_size, embed_size,
                          embeddings_ initializer="glorot_uniform",
                          input_length=1) )
context_model.add(Reshape( (embed_size, ) ) )

# merge
model = Sequential()
model.add (Merge([word_ model, context_ model], mode="dot") )
model.add(Dense(1, init="glorot_uniform", activation="sigmoid") )
model.compile(loss="mean_squared_error", optimizer="adam" )

Keras 提供的字符处理

In [9]:
from keras.preprocessing.text import *
from keras.preprocessing.sequence import skipgrams
text = "I love green eggs and ham ."

# 字符转换
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

# 创建字符表
word2id = tokenizer.word_index
id2word = {v:k for k,v in word2id.items()}

# 提取前10个字符对
wids = [word2id[w] for w in text_to_word_sequence(text)]
pairs, labels = skipgrams(wids, len(word2id))
print(len(pairs), len(labels))

for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
        id2word[pairs[i][0]], pairs[i][0],
        id2word[pairs[i][1] ], pairs[i][1],
        labels[i]))

56 56
(and (5), eggs (4)) -> 1
(and (5), green (3)) -> 0
(green (3), i (1)) -> 0
(eggs (4), and (5)) -> 0
(eggs (4), i (1)) -> 0
(ham (6), love (2)) -> 1
(love (2), green (3)) -> 1
(i (1), eggs (4)) -> 1
(love (2), eggs (4)) -> 1
(i (1), green (3)) -> 0


## CBOW

处理的流程如下：
![image](https://wx3.sinaimg.cn/large/69d4185bly1fyff3801sej20av0dodgn.jpg)

In [10]:
from keras.layers.core import Dense, Lambda
from keras.layers.embeddings import Embedding
from keras.models import Sequential
import keras.backend as K
vocab_size = 5000
embed_size = 300
window_size = 1

In [ ]:
model = Sequential()
model.add(Embedding (input_dim=vocab_ size, output_dim=embed_size,
                      embeddings_ initializer='glorot_uniform' ,
                      input_length=window_ size*2) )
model.add(Lambda (lambda x: K.mean(x, axis=1) , output_shape=(embed_size,)) )
model.add(Dense (vocab_size, kernel_initializer='glorot_uniform',activation='softmax') )
model.compile(loss='categorical_ crossentropy', optimizer="adam")

# RNN